![](http://sds.uanl.mx/wp-content/uploads/2019/11/logo-fcq-emblema.png)
# UNIVERSIDAD AUTONOMA DE NUEVO LEON
# FACULTAD DE CIENCIAS QUIMICAS

> ## *Producto Integrador de Aprendisaje*

### **Temas selectos de ingeniería**

### Integrantes:
* Victor Daniel Ponce Loya
* Luis Donaldo Tejeda Villarreal

**Objetivo del notebook:**

En este bloc de notas, exploremos el conjunto de datos dado y hagamos algunas inferencias en el camino. También finalmente vamos a construir un modelo de luz de línea de base gbm para empezar. 

**Objetivo de la competición:**

En esta competencia, tenemos el reto de analizar un conjunto de datos de cliente de Google Merchandise Store (también conocido como GStore, donde se vende el swag de Google) para predecir los ingresos por cliente. 

In [ ]:
import os
import json
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()

%matplotlib inline

from plotly import tools
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go


from sklearn import model_selection, preprocessing, metrics
import lightgbm as lgb

pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999

**Acerca del conjunto de datos:**

Al igual que la mayoría de las otras competiciones kaggle, se nos dan dos conjuntos de datos
* train.csv
* test.csv

Cada fila del conjunto de datos es una visita a la tienda. Estamos prediciendo el registro natural de la suma de todas las transacciones por usuario. 
    
Los campos de datos de los archivos especificados son 
* fullVisitorId- Identificador único para cada usuario de Google Merchandise Store..
* channelGrouping - El canal a través del cual el usuario llegó a la Tienda.
* date - La fecha en la que el usuario visitó la Tienda.
* device - Las especificaciones del dispositivo utilizado para acceder a la Tienda.
* geoNetwork - Esta sección contiene información sobre la geografía del usuario.
* sessionId - Un identificador único para esta visita a la tienda.
* socialEngagementType - Tipo de interacción, ya sea "Socialmente comprometido" o "No socialmente comprometido".
* totals - Esta sección contiene valores agregados en toda la sesión.
* trafficSource - Esta sección contiene información sobre la fuente de tráfico desde la que se originó la sesión.
* visitId - Identificador de esta sesión. Esto es parte del valor que normalmente se almacena como la cookie _utmb. Esto sólo es único para el usuario. Para un identificador completamente único, debe usar una combinación de fullVisitorId y visitId.
* visitNumber - El número de sesión para este usuario. Si esta es la primera sesión, después esto se fija a 1.
* visitStartTime - La marca de tiempo (expresada como tiempo POSIX).



In [ ]:
def load_df(csv_path='../input/ga-customer-revenue-prediction/train.csv', nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     nrows=nrows)
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df

In [ ]:
%%time
train_df = load_df()
test_df = load_df("../input/ga-customer-revenue-prediction/test.csv")

In [ ]:
train_df.head()

**Exploración variable de destino:**

Puesto que estamos prediciendo el registro natural de suma de todas las transacciones del usuario, vamos a resumir los ingresos de la transacción a nivel de usuario y tomar un registro y luego hacer una gráfica de dispersión.

In [ ]:
print("Exploración de la variable objetivo")

train_df["totals.transactionRevenue"] = train_df["totals.transactionRevenue"].astype('float')
gdf = train_df.groupby("fullVisitorId")["totals.transactionRevenue"].sum().reset_index()

plt.figure(figsize=(8,6))
plt.scatter(range(gdf.shape[0]), np.sort(np.log1p(gdf["totals.transactionRevenue"].values)))
plt.xlabel('index', fontsize=12)
plt.ylabel('Ingresos_por_transacciones', fontsize=12)
plt.show()

Wow, Esto confirma las dos primeras líneas de la visión general de la competencia.
    * La regla 80/20 ha demostrado ser cierta para muchas empresas: solo un pequeño porcentaje de clientes produce la mayor parte de los ingresos. Como tal, los equipos de marketing tienen el desafío de realizar inversiones apropiadas en estrategias promocionales.
De hecho, en este caso, la relación es aún menor.     

In [ ]:
nzi = pd.notnull(train_df["totals.transactionRevenue"]).sum()
nzr = (gdf["totals.transactionRevenue"]>0).sum()
print("Número de instancias en el conjunto de trenes con ingresos distintos de cero: ", nzi, " y la relación es: ", nzi / train_df.shape[0])
print("Número de clientes únicos con ingresos distintos de cero: ", nzr, "y la relación es: ", nzr / gdf.shape[0])

Así que la proporción de ingresos que generan clientes a clientes sin ingresos está en la relación os 1.3%

Dado que la mayoría de las filas tienen ingresos distintos de cero, en las siguientes gráficas vamos a echar un vistazo al recuento de cada categoría de la variable junto con el número de instancias donde los ingresos no son cero.

** Número de visitantes y visitantes comunes:**

Ahora veamos el número de visitantes únicos en el tren y el conjunto de pruebas y también el número de visitantes comunes.

In [ ]:
print("Número de visitantes únicos en el conjunto de trenes : ",train_df.fullVisitorId.nunique(), " fuera de las filas : ",train_df.shape[0])
print("Número de visitantes únicos en el conjunto de pruebas: ",test_df.fullVisitorId.nunique(), " fuera de las filas : ",test_df.shape[0])
print("Número de visitantes comunes en el tren y conjunto de pruebas : ",len(set(train_df.fullVisitorId.unique()).intersection(set(test_df.fullVisitorId.unique())) ))

**Columnas con valores constantes: **

Parece que hay bastantes características con valor constante en el conjunto de trenes. Vamos a obtener la lista de estas características. Como señala Svitlana en los comentarios a continuación, no incluyamos las columnas que tienen valor constante y algunos valores nulos. 

In [ ]:
const_cols = [c for c in train_df.columns if train_df[c].nunique(dropna=False)==1 ]
const_cols

Son bastantes. Dado que los valores son constantes, podemos simplemente eliminarlos de nuestra lista de características y ahorrar algo de memoria y tiempo en nuestro proceso de modelado. 

**Información del dispositivo:**

In [ ]:
def horizontal_bar_chart(cnt_srs, color):
    trace = go.Bar(
        y=cnt_srs.index[::-1],
        x=cnt_srs.values[::-1],
        showlegend=False,
        orientation = 'h',
        marker=dict(
            color=color,
        ),
    )
    return trace

# Device Browser
cnt_srs = train_df.groupby('device.browser')['totals.transactionRevenue'].agg(['size', 'count', 'mean'])
cnt_srs.columns = ["count", "count of non-zero revenue", "mean"]
cnt_srs = cnt_srs.sort_values(by="count", ascending=False)
trace1 = horizontal_bar_chart(cnt_srs["count"].head(10), 'rgba(50, 171, 96, 0.6)')
trace2 = horizontal_bar_chart(cnt_srs["count of non-zero revenue"].head(10), 'rgba(50, 171, 96, 0.6)')
trace3 = horizontal_bar_chart(cnt_srs["mean"].head(10), 'rgba(50, 171, 96, 0.6)')

# Device Category
cnt_srs = train_df.groupby('device.deviceCategory')['totals.transactionRevenue'].agg(['size', 'count', 'mean'])
cnt_srs.columns = ["count", "count of non-zero revenue", "mean"]
cnt_srs = cnt_srs.sort_values(by="count", ascending=False)
trace4 = horizontal_bar_chart(cnt_srs["count"].head(10), 'rgba(71, 58, 131, 0.8)')
trace5 = horizontal_bar_chart(cnt_srs["count of non-zero revenue"].head(10), 'rgba(71, 58, 131, 0.8)')
trace6 = horizontal_bar_chart(cnt_srs["mean"].head(10), 'rgba(71, 58, 131, 0.8)')

# Operating system
cnt_srs = train_df.groupby('device.operatingSystem')['totals.transactionRevenue'].agg(['size', 'count', 'mean'])
cnt_srs.columns = ["count", "count of non-zero revenue", "mean"]
cnt_srs = cnt_srs.sort_values(by="count", ascending=False)
trace7 = horizontal_bar_chart(cnt_srs["count"].head(10), 'rgba(246, 78, 139, 0.6)')
trace8 = horizontal_bar_chart(cnt_srs["count of non-zero revenue"].head(10),'rgba(246, 78, 139, 0.6)')
trace9 = horizontal_bar_chart(cnt_srs["mean"].head(10),'rgba(246, 78, 139, 0.6)')

# Creating two subplots
fig = tools.make_subplots(rows=3, cols=3, vertical_spacing=0.04, 
                          subplot_titles=["Navegador de dispositivos - Recuento", "Navegador de dispositivos - Recuento de ingresos distintos de cero", "Navegador de dispositivos - Ingresos medios",
                                          "Categoría del dispositivo - Recuento",  "Categoría de dispositivo - Recuento de ingresos distintos de cero", "Categoría del dispositivo - Ingresos medios", 
                                          "Sistema operativo del dispositivo - Recuento", "Sistema operativo para dispositivos : recuento de ingresos distintos de cero", "Sistema operativo del dispositivo - Ingresos medios"])

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)
fig.append_trace(trace3, 1, 3)
fig.append_trace(trace4, 2, 1)
fig.append_trace(trace5, 2, 2)
fig.append_trace(trace6, 2, 3)
fig.append_trace(trace7, 3, 1)
fig.append_trace(trace8, 3, 2)
fig.append_trace(trace9, 3, 3)

fig['layout'].update(height=1200, width=1200, paper_bgcolor='rgb(233,233,233)', title="Gráficas de dispositivos")
py.iplot(fig, filename='device-plots')

Inferencias:
* La distribución del navegador de dispositivos es similar tanto en el recuento como en el recuento de parcelas de ingresos distintos de cero
* En la categoría de dispositivo frontal, el escritorio parece tener un mayor porcentaje de recuentos de ingresos distintos de cero en comparación con los dispositivos móviles.
* En el sistema operativo del dispositivo, aunque el número de recuentos es más de Windows, el número de recuentos donde los ingresos no son cero es más para Macintosh.
* Chrome OS también tiene un mayor porcentaje de recuentos de ingresos distintos de cero
* En el lado del sistema operativo móvil, iOS tiene más porcentaje de recuentos de ingresos distintos de cero en comparación con Android 

**Exploración de fecha:**

In [ ]:
import datetime

def scatter_plot(cnt_srs, color):
    trace = go.Scatter(
        x=cnt_srs.index[::-1],
        y=cnt_srs.values[::-1],
        showlegend=False,
        marker=dict(
            color=color,
        ),
    )
    return trace

train_df['date'] = train_df['date'].apply(lambda x: datetime.date(int(str(x)[:4]), int(str(x)[4:6]), int(str(x)[6:])))
cnt_srs = train_df.groupby('date')['totals.transactionRevenue'].agg(['size', 'count'])
cnt_srs.columns = ["count", "count of non-zero revenue"]
cnt_srs = cnt_srs.sort_index()
#cnt_srs.index = cnt_srs.index.astype('str')
trace1 = scatter_plot(cnt_srs["count"], 'red')
trace2 = scatter_plot(cnt_srs["count of non-zero revenue"], 'blue')

fig = tools.make_subplots(rows=2, cols=1, vertical_spacing=0.08,
                          subplot_titles=["Fecha - Recuento", "Fecha - Número de ingresos no nulos"])
fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 2, 1)
fig['layout'].update(height=800, width=800, paper_bgcolor='rgb(233,233,233)', title="Gráficas de fecha")
py.iplot(fig, filename='date-plots')

Inferencias:
* Tenemos datos del 1 de agosto de 2016 al 31 de julio de 2017 en nuestro conjunto de datos de formación
* En noviembre de 2016, aunque hay un aumento en el número de visitantes, no hay aumento en el número de ingresos distintos de cero durante ese período de tiempo (en relación con la media).

In [ ]:
test_df['date'] = test_df['date'].apply(lambda x: datetime.date(int(str(x)[:4]), int(str(x)[4:6]), int(str(x)[6:])))
cnt_srs = test_df.groupby('date')['fullVisitorId'].size()


trace = scatter_plot(cnt_srs, 'red')

layout = go.Layout(
    height=400,
    width=800,
    paper_bgcolor='rgb(233,233,233)',
    title='Fechas en el conjunto de pruebas'
)

data = [trace]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename="ActivationDate")

En el conjunto de pruebas, tenemos fechas del 2 de agosto de 2017 al 30 de abril de 2018. Así que no hay fechas comunes entre el tren y el conjunto de pruebas. Por lo tanto, podría ser una buena idea realizar la validación basada en el tiempo para este conjunto de datos.

**Información geográfica:**

In [ ]:
# Continent
cnt_srs = train_df.groupby('geoNetwork.continent')['totals.transactionRevenue'].agg(['size', 'count', 'mean'])
cnt_srs.columns = ["count", "count of non-zero revenue", "mean"]
cnt_srs = cnt_srs.sort_values(by="count", ascending=False)
trace1 = horizontal_bar_chart(cnt_srs["count"].head(10), 'rgba(58, 71, 80, 0.6)')
trace2 = horizontal_bar_chart(cnt_srs["count of non-zero revenue"].head(10), 'rgba(58, 71, 80, 0.6)')
trace3 = horizontal_bar_chart(cnt_srs["mean"].head(10), 'rgba(58, 71, 80, 0.6)')

# Sub-continent
cnt_srs = train_df.groupby('geoNetwork.subContinent')['totals.transactionRevenue'].agg(['size', 'count', 'mean'])
cnt_srs.columns = ["count", "count of non-zero revenue", "mean"]
cnt_srs = cnt_srs.sort_values(by="count", ascending=False)
trace4 = horizontal_bar_chart(cnt_srs["count"], 'orange')
trace5 = horizontal_bar_chart(cnt_srs["count of non-zero revenue"], 'orange')
trace6 = horizontal_bar_chart(cnt_srs["mean"], 'orange')

# Network domain
cnt_srs = train_df.groupby('geoNetwork.networkDomain')['totals.transactionRevenue'].agg(['size', 'count', 'mean'])
cnt_srs.columns = ["count", "count of non-zero revenue", "mean"]
cnt_srs = cnt_srs.sort_values(by="count", ascending=False)
trace7 = horizontal_bar_chart(cnt_srs["count"].head(10), 'blue')
trace8 = horizontal_bar_chart(cnt_srs["count of non-zero revenue"].head(10), 'blue')
trace9 = horizontal_bar_chart(cnt_srs["mean"].head(10), 'blue')

# Creating two subplots
fig = tools.make_subplots(rows=3, cols=3, vertical_spacing=0.08, horizontal_spacing=0.15, 
                          subplot_titles=["Continente - Recuento", "Continente - Recuento de ingresos no cero", "Continente - Ingresos medios",
                                          "Subcontinente - Recuento",  "Subcontinente - Recuento de Ingresos No Cero", "Subcontinente - Ingresos medios",
                                          "Dominio de red - Recuento", "Dominio de red - Recuento de ingresos no nulo", "Dominio de red - Ingresos medios"])

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)
fig.append_trace(trace3, 1, 3)
fig.append_trace(trace4, 2, 1)
fig.append_trace(trace5, 2, 2)
fig.append_trace(trace6, 2, 3)
fig.append_trace(trace7, 3, 1)
fig.append_trace(trace8, 3, 2)
fig.append_trace(trace9, 3, 3)

fig['layout'].update(height=1500, width=1200, paper_bgcolor='rgb(233,233,233)', title="Parcelas de geografía")
py.iplot(fig, filename='geo-plots')

Inferencias:
* En la trama del continente, podemos ver que Estados Unidos tiene tanto un mayor número de recuentos, así como el mayor número de recuentos donde los ingresos son distintos de cero
* Aunque Asia y Europa tienen un alto número de recuentos, el número de recuentos de ingresos distintos de cero de estos continentes es comparativamente bajo. 
* Podemos inferir los dos primeros puntos de la parcela de los subcontinentes también.
* Si el dominio de red es "unknown.unknown" en lugar de "(no establecido)", entonces el número de recuentos con ingresos distintos de cero tiende a ser menor. 

**Fuente de tráfico:**


In [ ]:
# Continent
cnt_srs = train_df.groupby('trafficSource.source')['totals.transactionRevenue'].agg(['size', 'count', 'mean'])
cnt_srs.columns = ["count", "count of non-zero revenue", "mean"]
cnt_srs = cnt_srs.sort_values(by="count", ascending=False)
trace1 = horizontal_bar_chart(cnt_srs["count"].head(10), 'green')
trace2 = horizontal_bar_chart(cnt_srs["count of non-zero revenue"].head(10), 'green')
trace3 = horizontal_bar_chart(cnt_srs["mean"].head(10), 'green')

# Sub-continent
cnt_srs = train_df.groupby('trafficSource.medium')['totals.transactionRevenue'].agg(['size', 'count', 'mean'])
cnt_srs.columns = ["count", "count of non-zero revenue", "mean"]
cnt_srs = cnt_srs.sort_values(by="count", ascending=False)
trace4 = horizontal_bar_chart(cnt_srs["count"], 'purple')
trace5 = horizontal_bar_chart(cnt_srs["count of non-zero revenue"], 'purple')
trace6 = horizontal_bar_chart(cnt_srs["mean"], 'purple')

# Creating two subplots
fig = tools.make_subplots(rows=2, cols=3, vertical_spacing=0.08, horizontal_spacing=0.15, 
                          subplot_titles=["Fuente de tráfico - Recuento", "Fuente de tráfico - Recuento de ingresos no nulo", "Fuente de Tráfico - Ingresos Medios",
                                          "Medio de origen de tráfico - Recuento",  "Medio de origen de tráfico - Recuento de ingresos no nulo", "Medio de origen de tráfico - Ingresos medios"
                                          ])

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)
fig.append_trace(trace3, 1, 3)
fig.append_trace(trace4, 2, 1)
fig.append_trace(trace5, 2, 2)
fig.append_trace(trace6, 2, 3)

fig['layout'].update(height=1000, width=1200, paper_bgcolor='rgb(233,233,233)', title="Gráficas de fuentes de tráfico")
py.iplot(fig, filename='traffic-source-plots')

Inferencias:
* En la gráfica de origen de tráfico, aunque Youtube tiene un gran número de recuentos en el conjunto de datos, el número de recuentos de ingresos distintos de cero es muy menor. 
* Google plex tiene una alta proporción de número de ingresos distintos de cero a la cuenta total en la gráfica de origen de tráfico. 
* En el medio de origen de tráfico, "referencia" tiene más número de ingresos distintos de cero en comparación con el medio "orgánico".

**Perfil del visitante:**

Ahora veamos las variables de perfil de visitante como el número de páginas vistas por el visitante, el número de visitas del visitante y ver cómo se ven.

In [ ]:

# Page views
cnt_srs = train_df.groupby('totals.pageviews')['totals.transactionRevenue'].agg(['size', 'count', 'mean'])
cnt_srs.columns = ["count", "count of non-zero revenue", "mean"]
cnt_srs = cnt_srs.sort_values(by="count", ascending=False)
trace1 = horizontal_bar_chart(cnt_srs["count"].head(60), 'cyan')
trace2 = horizontal_bar_chart(cnt_srs["count of non-zero revenue"].head(60), 'cyan')
trace5 = horizontal_bar_chart(cnt_srs["mean"].head(60), 'cyan')

# Hits
cnt_srs = train_df.groupby('totals.hits')['totals.transactionRevenue'].agg(['size', 'count', 'mean'])
cnt_srs.columns = ["count", "count of non-zero revenue", 'mean']
cnt_srs = cnt_srs.sort_values(by="count", ascending=False)
trace3 = horizontal_bar_chart(cnt_srs["count"].head(60), 'black')
trace4 = horizontal_bar_chart(cnt_srs["count of non-zero revenue"].head(60), 'black')
trace6 = horizontal_bar_chart(cnt_srs["mean"].head(60), 'black')

# Creating two subplots
fig = tools.make_subplots(rows=2, cols=3, vertical_spacing=0.08, horizontal_spacing=0.15, 
                          subplot_titles=["Total de páginas vistas - Recuento", "Total de páginas vistas - Recuento de ingresos distintos de cero", "Total de páginas vistas - Ingresos medios",
                                          "Total de aciertos - Recuento",  "Total de aciertos - Recuento de ingresos no nulo", "Total de aciertos - Ingresos medios"])

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)
fig.append_trace(trace5, 1, 3)
fig.append_trace(trace3, 2, 1)
fig.append_trace(trace4, 2, 2)
fig.append_trace(trace6, 2, 3)

fig['layout'].update(height=1200, width=900, paper_bgcolor='rgb(233,233,233)', title="Gráficas de perfil de visitantes")
py.iplot(fig, filename='visitor-profile-plots')

Inferencias:
* Ambas variables parecen muy predictivas
* La gráfica de recuento muestra la naturaleza decreciente, es decir, tenemos un recuento total muy alto para menos número de visitas y vistas de página por transacción de visitante y el recuento general disminuye cuando aumenta el número de visitas por transacción de visitante.
* Por otro lado, podemos ver claramente que cuando aumenta el número de visitas / páginas vistas por transacción de visitante, vemos que hay un gran número de recuentos de ingresos distintos de cero. 

**Modelo de línea base:**

Ahora vamos a crear un modelo de línea base en este conjunto de datos. Antes de empezar a crear modelos, veamos los nombres de variables que hay en el conjunto de datos train y no en el conjunto de datos de prueba. 

In [ ]:
print("Variables no en prueba, sino en tren : ", set(train_df.columns).difference(set(test_df.columns)))

Por lo tanto, aparte de la variable de destino, hay una variable más "trafficSource.campaignCode" que no está presente en el conjunto de datos de prueba. Así que tenemos que eliminar esta variable al construir modelos. También podemos soltar las variables constantes que obtuvimos antes.

También podemos quitar el "sessionId" ya que es un identificador único de la visita.

In [ ]:
cols_to_drop = const_cols + ['sessionId']

train_df = train_df.drop(cols_to_drop + ["trafficSource.campaignCode"], axis=1)
test_df = test_df.drop(cols_to_drop, axis=1)

Ahora vamos a crear divisiones de desarrollo y validación basadas en el tiempo para crear el modelo. Podemos tomar los últimos dos meses como muestra de validación.

In [ ]:
# Impute 0 for missing target values
train_df["totals.transactionRevenue"].fillna(0, inplace=True)
train_y = train_df["totals.transactionRevenue"].values
train_id = train_df["fullVisitorId"].values
test_id = test_df["fullVisitorId"].values


# label encode the categorical variables and convert the numerical variables to float
cat_cols = ["channelGrouping", "device.browser", 
            "device.deviceCategory", "device.operatingSystem", 
            "geoNetwork.city", "geoNetwork.continent", 
            "geoNetwork.country", "geoNetwork.metro",
            "geoNetwork.networkDomain", "geoNetwork.region", 
            "geoNetwork.subContinent", "trafficSource.adContent", 
            "trafficSource.adwordsClickInfo.adNetworkType", 
            "trafficSource.adwordsClickInfo.gclId", 
            "trafficSource.adwordsClickInfo.page", 
            "trafficSource.adwordsClickInfo.slot", "trafficSource.campaign",
            "trafficSource.keyword", "trafficSource.medium", 
            "trafficSource.referralPath", "trafficSource.source",
            'trafficSource.adwordsClickInfo.isVideoAd', 'trafficSource.isTrueDirect']
for col in cat_cols:
    print(col)
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(train_df[col].values.astype('str')) + list(test_df[col].values.astype('str')))
    train_df[col] = lbl.transform(list(train_df[col].values.astype('str')))
    test_df[col] = lbl.transform(list(test_df[col].values.astype('str')))


num_cols = ["totals.hits", "totals.pageviews", "visitNumber", "visitStartTime", 'totals.bounces',  'totals.newVisits']    
for col in num_cols:
    train_df[col] = train_df[col].astype(float)
    test_df[col] = test_df[col].astype(float)

# Split the train dataset into development and valid based on time 
dev_df = train_df[train_df['date']<=datetime.date(2017,5,31)]
val_df = train_df[train_df['date']>datetime.date(2017,5,31)]
dev_y = np.log1p(dev_df["totals.transactionRevenue"].values)
val_y = np.log1p(val_df["totals.transactionRevenue"].values)

dev_X = dev_df[cat_cols + num_cols] 
val_X = val_df[cat_cols + num_cols] 
test_X = test_df[cat_cols + num_cols] 

In [ ]:
# custom function to run light gbm model
def run_lgb(train_X, train_y, val_X, val_y, test_X):
    params = {
        "objective" : "regression",
        "metric" : "rmse", 
        "num_leaves" : 30,
        "min_child_samples" : 100,
        "learning_rate" : 0.1,
        "bagging_fraction" : 0.7,
        "feature_fraction" : 0.5,
        "bagging_frequency" : 5,
        "bagging_seed" : 2018,
        "verbosity" : -1
    }
    
    lgtrain = lgb.Dataset(train_X, label=train_y)
    lgval = lgb.Dataset(val_X, label=val_y)
    model = lgb.train(params, lgtrain, 1000, valid_sets=[lgval], early_stopping_rounds=100, verbose_eval=100)
    
    pred_test_y = model.predict(test_X, num_iteration=model.best_iteration)
    pred_val_y = model.predict(val_X, num_iteration=model.best_iteration)
    return pred_test_y, model, pred_val_y

# Training the model #
pred_test, model, pred_val = run_lgb(dev_X, dev_y, val_X, val_y, test_X)

Ahora vamos a calcular la métrica de evaluación en los datos de validación como se menciona en [este nuevo hilo de discusión](https://www.kaggle.com/c/ga-customer-revenue-prediction/discussion/66737). Así que tenemos que hacer una suma para todas las transacciones del usuario y luego hacer una transformación de registro en la parte superior. También hagamos que los valores sean inferiores a 0 a 0, ya que los ingresos de transacción solo pueden ser 0 o más. 

In [ ]:
from sklearn import metrics
pred_val[pred_val<0] = 0
val_pred_df = pd.DataFrame({"fullVisitorId":val_df["fullVisitorId"].values})
val_pred_df["transactionRevenue"] = val_df["totals.transactionRevenue"].values
val_pred_df["PredictedRevenue"] = np.expm1(pred_val)
#print(np.sqrt(metrics.mean_squared_error(np.log1p(val_pred_df["transactionRevenue"].values), np.log1p(val_pred_df["PredictedRevenue"].values))))
val_pred_df = val_pred_df.groupby("fullVisitorId")["transactionRevenue", "PredictedRevenue"].sum().reset_index()
print(np.sqrt(metrics.mean_squared_error(np.log1p(val_pred_df["transactionRevenue"].values), np.log1p(val_pred_df["PredictedRevenue"].values))))

Así que estamos obteniendo una puntuación de validación de 1.70** usando este método contra la puntuación de la tabla de clasificación pública de 1.44**. Así que por favor tenga cuidado mientras trata con esto. 

Ahora vamos a preparar el archivo de envío similar al conjunto de validación.

In [ ]:
sub_df = pd.DataFrame({"fullVisitorId":test_id})
pred_test[pred_test<0] = 0
sub_df["PredictedLogRevenue"] = np.expm1(pred_test)
sub_df = sub_df.groupby("fullVisitorId")["PredictedLogRevenue"].sum().reset_index()
sub_df.columns = ["fullVisitorId", "PredictedLogRevenue"]
sub_df["PredictedLogRevenue"] = np.log1p(sub_df["PredictedLogRevenue"])
sub_df.to_csv("baseline_lgb.csv", index=False)

In [ ]:
sub_df.head()

**Importancia de la característica:**

Ahora echemos un vistazo a las características importantes del modelo de luz gbm.

In [ ]:
fig, ax = plt.subplots(figsize=(12,18))
lgb.plot_importance(model, max_num_features=50, height=0.8, ax=ax)
ax.grid(False)
plt.title("LightGBM - Importancia de las funciones", fontsize=15)
plt.ylabel("Funciones")
plt.xlabel("Importancia de la característica")
plt.show()

"totals.pageviews" resulta ser la característica más importante seguida de "totals.hits" y "visitStartTime". 

**Más por venir. ¡Estén atentos! **